## 08 实现我们自己的 Linear Regression

### 一元线性回归
![IMAGE](https://farm2.staticflickr.com/1736/27812680287_f3390e64a7_o.png)

### 多元线性回归
x 从一个点，变成一个向量
![IMAGE](https://farm2.staticflickr.com/1748/40872266130_d6cd7cac7f_o.png)

一元线性回归，我们参数是 a 和 b

多元线性回归，参数是$\theta_{0}\theta_{1}\theta_{2}...\theta_{n}$

### 多元线性回归的目标
**针对第 i 个样本：**
![IMAGE](https://farm2.staticflickr.com/1725/41781668805_e7da0b997d_o.png)
> $\theta$原本是一个行向量，转置后变成一个列向量，放在右边，等待与数据进行点乘

![IMAGE](https://farm2.staticflickr.com/1748/41781728685_4c4e2c65e8_o.png)

**推广到所有样本**
> $X_{b}$ 和 $X$的区别是多了 一列1

![IMAGE](https://farm2.staticflickr.com/1731/28808265538_2fcc1e7731_o.png)

对比
![IMAGE](https://farm2.staticflickr.com/1723/42006651364_59291395b3_o.png)

### 目标函数的向量化
![IMAGE](https://farm2.staticflickr.com/1729/28808353058_ed0cf16e8f_o.png)
最终得到的是一个数字

### 对目标函数求导
![IMAGE](https://farm2.staticflickr.com/1723/27812966087_816d2295a6_o.png)


```python
# 左边添加1列
X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
# ❤️❤️ 核心计算
self._theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_train)
```

![IMAGE](https://farm2.staticflickr.com/1751/40872558980_844af755fd_o.png)
![IMAGE](https://farm2.staticflickr.com/1760/27813082817_fc588e0846_o.png)
* 缺点：这里的 n 是对行和列的一个普遍指代。如果数据量很高或者 feature 很多，都会导致时间时间复杂度很高，解决方法可以利用下一章的知识。实际情况中直接使用这种方法进行运算的很少很少。
* 优点：因为 theta 是 x 的系数，没有量纲，所以不用进行归一化处理。

![IMAGE](https://farm2.staticflickr.com/1727/40872651190_b92d082c52_o.png)

有时候我们把截距和系数分别汇报给用户，并且系数可以反映不同 feature 的贡献程度。

### 使用我们自己制作 Linear Regression

代码参见 [这里](playML/LinearRegression.py)
```python
class LinearRegression:

    def __init__(self):
        """初始化Linear Regression模型"""
        self.coef_ = None # theta 0 
        self.intercept_ = None # theta 1~n
        self._theta = None # 这是一个私有的变量, theta0~n

    def fit_normal(self, X_train, y_train):
        """根据训练数据集X_train, y_train训练Linear Regression模型"""
        assert X_train.shape[0] == y_train.shape[0], \
            "the size of X_train must be equal to the size of y_train"
        # 左边添加1列
        X_b = np.hstack([np.ones((len(X_train), 1)), X_train])
        # ❤️❤️ 核心计算
        self._theta = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y_train)

        self.intercept_ = self._theta[0]
        self.coef_ = self._theta[1:]

        return self

    def predict(self, X_predict):
        """给定待预测数据集X_predict，返回表示X_predict的结果向量"""
        assert self.intercept_ is not None and self.coef_ is not None, \
            "must fit before predict!"
        assert X_predict.shape[1] == len(self.coef_), \
            "the feature number of X_predict must be equal to X_train"
            
            
        # ❤️ 先补充一列
        X_b = np.hstack([np.ones((len(X_predict), 1)), X_predict])
        # ❤️ 点乘原来我们计算出来的 theta
        return X_b.dot(self._theta)

    def score(self, X_test, y_test):
        """根据测试数据集 X_test 和 y_test 确定当前模型的准确度"""

        y_predict = self.predict(X_test)
        return r2_score(y_test, y_predict)

    def __repr__(self):
        return "LinearRegression()"


```

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

In [3]:
X.shape

(490, 13)

In [4]:
from playML.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, seed=666)

In [5]:
from playML.LinearRegression import LinearRegression

reg = LinearRegression()
# 这回我们使用全部X 中的列
reg.fit_normal(X_train, y_train)

LinearRegression()

In [6]:
reg.coef_

array([ -1.18919477e-01,   3.63991462e-02,  -3.56494193e-02,
         5.66737830e-02,  -1.16195486e+01,   3.42022185e+00,
        -2.31470282e-02,  -1.19509560e+00,   2.59339091e-01,
        -1.40112724e-02,  -8.36521175e-01,   7.92283639e-03,
        -3.81966137e-01])

In [7]:
reg.intercept_

34.161435496224712

In [8]:
reg.score(X_test, y_test)

0.81298026026584658